## Code for replicating gao et al research on VUA Sequence Model

In [1]:
# mount drive
from google.colab import drive
ROOT = '/content/drive'
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# add repo directory to path
import os
import sys
from os.path import join 
repo_dir = '/content/drive/MyDrive/Repos/metaphor-detection'
if repo_dir not in sys.path:
    sys.path.append(repo_dir)
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Repos/metaphor-detection']


In [3]:
# directories
# to download glove and elmo vectors see: notebooks/Download_large_data.ipynb
data_dir = repo_dir + '/resources/metaphor-in-context/data/'
glove_dir = repo_dir + '/resources/glove/'
elmo_dir = repo_dir + '/resources/elmo/'

In [11]:

%pwd

'/content/drive/MyDrive/Repos/metaphor-detection'

Gao code

In [5]:
# pip install requirements (takes a while)
!cd drive/MyDrive/Repos/metaphor-detection/; pip install -r gao-g-requirements.txt
!pip install --upgrade google-cloud-storage

In [75]:
from core.gao_files.sequence.util import get_num_lines, get_pos2idx_idx2pos, index_sequence, get_vocab, embed_indexed_sequence, \
    get_word2idx_idx2word, get_embedding_matrix, write_predictions, get_performance_VUAverb_val, \
    get_performance_VUAverb_test, get_performance_VUA_test
from core.gao_files.sequence.util import TextDatasetWithGloveElmoSuffix as TextDataset
from core.gao_files.sequence.util import evaluate
from core.gao_files.sequence.model import RNNSequenceModel

#!pip install Ipython --upgrade

%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

import csv
import h5py
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import ast

In [13]:
print("PyTorch version:")
print(torch.__version__)
print("GPU Detected:")
print(torch.cuda.is_available())
using_GPU = torch.cuda.is_available()

PyTorch version:
1.10.0+cu111
GPU Detected:
True


In [14]:
"""
1. Data pre-processing
"""
'''
1.1 VUA
get raw dataset as a list:
  Each element is a triple:
    a sentence: string
    a list of labels: 
    a list of pos: 
'''
pos_set = set()
raw_train_vua = []
with open(data_dir + 'VUAsequence/VUA_seq_formatted_train.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        pos_seq = ast.literal_eval(line[4])
        label_seq = ast.literal_eval(line[3])
        assert (len(pos_seq) == len(label_seq))
        assert (len(line[2].split()) == len(pos_seq))
        raw_train_vua.append([line[2], label_seq, pos_seq])
        pos_set.update(pos_seq)

raw_val_vua = []
with open(data_dir + 'VUAsequence/VUA_seq_formatted_val.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        pos_seq = ast.literal_eval(line[4])
        label_seq = ast.literal_eval(line[3])
        assert (len(pos_seq) == len(label_seq))
        assert (len(line[2].split()) == len(pos_seq))
        raw_val_vua.append([line[2], label_seq, pos_seq])
        pos_set.update(pos_seq)

# embed the pos tags
pos2idx, idx2pos = get_pos2idx_idx2pos(pos_set)

for i in range(len(raw_train_vua)):
    raw_train_vua[i][2] = index_sequence(pos2idx, raw_train_vua[i][2])
for i in range(len(raw_val_vua)):
    raw_val_vua[i][2] = index_sequence(pos2idx, raw_val_vua[i][2])
print('size of training set, validation set: ', len(raw_train_vua), len(raw_val_vua))

size of training set, validation set:  6323 1550


In [15]:
raw_train_vua[:4]

[["Ca n't fail to be entertaining .",
  [0, 0, 0, 0, 0, 0, 0],
  [14, 7, 14, 4, 14, 5, 2]],
 ['How much was he going to tell her ?',
  [0, 0, 0, 0, 0, 0, 0, 0, 0],
  [7, 5, 14, 9, 14, 4, 14, 9, 2]],
 ['Up until that news hit the Committee , Don had won the day with his UK Vehicle Division proposals .',
  [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  [12, 12, 6, 10, 14, 6, 1, 2, 1, 14, 14, 6, 10, 12, 5, 1, 1, 1, 10, 2]],
 ["Could go on to the rugby and go with them could n't he ?",
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [14, 14, 4, 12, 6, 10, 3, 14, 12, 9, 14, 7, 9, 2]]]

In [16]:
"""
2. Data preparation
"""
'''
2. 1
get vocabulary and glove embeddings in raw dataset 
'''
# vocab is a set of words
vocab = get_vocab(raw_train_vua)
# two dictionaries. <PAD>: 0, <UNK>: 1
word2idx, idx2word = get_word2idx_idx2word(vocab)
# glove_embeddings a nn.Embeddings
glove_embeddings = get_embedding_matrix(glove_dir + 'glove.840B.300d.txt',word2idx, idx2word, normalization=False)
# elmo_embeddings
elmos_train_vua = h5py.File(elmo_dir + 'VUA_train.hdf5', 'r')
elmos_val_vua = h5py.File(elmo_dir + 'VUA_val.hdf5', 'r')
# no suffix embeddings for sequence labeling
suffix_embeddings = None

vocab size:  13843


100%|██████████| 2196017/2196017 [01:05<00:00, 33750.95it/s]


Number of pre-trained word vectors loaded:  13404
Embeddings mean:  0.0005707233212888241
Embeddings stdev:  0.3729434907436371


In [17]:
print(len(vocab))
glove_embeddings.weight.shape
# 300d embeddings for the 13843 words in the vocab

13843


torch.Size([13845, 300])

In [18]:
'''
2. 2
embed the datasets
'''
# raw_train_vua: sentence, label_seq, pos_seq
# embedded_train_vua: embedded_sentence, pos, labels
embedded_train_vua = [[embed_indexed_sequence(example[0], example[2], word2idx,
                                      glove_embeddings, elmos_train_vua, suffix_embeddings),
                       example[2], example[1]]
                      for example in raw_train_vua]
embedded_val_vua = [[embed_indexed_sequence(example[0], example[2], word2idx,
                                    glove_embeddings, elmos_val_vua, suffix_embeddings),
                     example[2], example[1]]
                    for example in raw_val_vua]

In [19]:
# embedded_train_vua is a list of lists -- one list per sentence
# each sentence list contains 
#     an array of embeddings (seq_length x embedding_dim)
#     list of pos tag ids
#     list of labels
print(len(embedded_train_vua))
print(len(embedded_train_vua[0]))
print(embedded_train_vua[0][0].shape)
print(embedded_train_vua[0][1])
print(embedded_train_vua[0][2])

6323
3
(7, 1324)
[14, 7, 14, 4, 14, 5, 2]
[0, 0, 0, 0, 0, 0, 0]


In [20]:
'''
2. 3
set up Dataloader for batching
'''
# Separate the input (embedded_sequence) and labels in the indexed train sets.
# embedded_train_vua: embedded_sentence, pos, labels
train_dataset_vua = TextDataset([example[0] for example in embedded_train_vua],
                                [example[1] for example in embedded_train_vua],
                                [example[2] for example in embedded_train_vua])
val_dataset_vua = TextDataset([example[0] for example in embedded_val_vua],
                              [example[1] for example in embedded_val_vua],
                              [example[2] for example in embedded_val_vua])

# Data-related hyperparameters
batch_size = 64
# Set up a DataLoader for the training, validation, and test dataset
train_dataloader_vua = DataLoader(dataset=train_dataset_vua, batch_size=batch_size, shuffle=True,
                              collate_fn=TextDataset.collate_fn)
val_dataloader_vua = DataLoader(dataset=val_dataset_vua, batch_size=batch_size,
                            collate_fn=TextDataset.collate_fn)

In [21]:
"""
3. Model training
"""
'''
3. 1 
set up model, loss criterion, optimizer
'''
# Instantiate the model
# embedding_dim = glove + elmo + suffix indicator
# dropout1: dropout on input to RNN
# dropout2: dropout in RNN; would be used if num_layers!=1
# dropout3: dropout on hidden state of RNN to linear layer
RNNseq_model = RNNSequenceModel(num_classes=2, embedding_dim=300 + 1024, hidden_size=300, num_layers=1, bidir=True,
                                dropout1=0.5, dropout2=0, dropout3=0.1)
# Move the model to the GPU if available
if using_GPU:
    RNNseq_model = RNNseq_model.cuda()
# Set up criterion for calculating loss
loss_criterion = nn.NLLLoss()
# Set up an optimizer for updating the parameters of the rnn_clf
rnn_optimizer = optim.Adam(RNNseq_model.parameters(), lr=0.005)
# Number of epochs (passes through the dataset) to train the model for.
num_epochs = 10

In [22]:
'''
3. 2
train model
'''
train_loss = []
val_loss = []
performance_matrix = None
val_f1s = []
train_f1s = []
# A counter for the number of gradient updates
num_iter = 0
comparable = []
for epoch in range(num_epochs):
    print("Starting epoch {}".format(epoch + 1))
    for (__, example_text, example_lengths, labels) in train_dataloader_vua:
        example_text = Variable(example_text)
        example_lengths = Variable(example_lengths)
        labels = Variable(labels)
        if using_GPU:
            example_text = example_text.cuda()
            example_lengths = example_lengths.cuda()
            labels = labels.cuda()
        # predicted shape: (batch_size, seq_len, 2)
        predicted = RNNseq_model(example_text, example_lengths)
        batch_loss = loss_criterion(predicted.view(-1, 2), labels.view(-1))
        rnn_optimizer.zero_grad()
        batch_loss.backward()
        rnn_optimizer.step()
        num_iter += 1
        # Calculate validation and training set loss and accuracy every 200 gradient updates
        if num_iter % 200 == 0:
            avg_eval_loss, performance_matrix = evaluate(idx2pos, val_dataloader_vua, RNNseq_model,
                                                         loss_criterion, using_GPU)
            val_loss.append(avg_eval_loss)
            val_f1s.append(performance_matrix[:, 2])
            print("Iteration {}. Validation Loss {}.".format(num_iter, avg_eval_loss))
#             avg_eval_loss, performance_matrix = evaluate(idx2pos, train_dataloader_vua, RNNseq_model,
#                                                          loss_criterion, using_GPU)
#             train_loss.append(avg_eval_loss)
#             train_f1s.append(performance_matrix[:, 2])
#             print("Iteration {}. Training Loss {}.".format(num_iter, avg_eval_loss))

Starting epoch 1
Starting epoch 2
Starting epoch 3


/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:217: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_text = Variable(eval_text, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:218: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_lengths = Variable(eval_lengths, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:219: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_labels = Variable(eval_labels, volatile=True)


------------------------------
total_eval_loss.shape torch.Size([])
PRFA performance for  SYM nan nan nan 100.0
PRFA performance for  PROPN 100.0 4.545454545454546 8.695652173913043 98.87940234791888
PRFA performance for  PUNCT 100.0 60.0 75.0 99.94811932555123
PRFA performance for  CCONJ nan nan nan 100.0
PRFA performance for  PART 71.9298245614035 42.2680412371134 53.24675324675324 93.66754617414249
PRFA performance for  ADJ 74.67532467532467 33.43023255813954 46.18473895582329 91.65888577653284
PRFA performance for  DET 85.92592592592592 92.8 89.23076923076921 98.41852584015815
PRFA performance for  ADV 77.27272727272727 33.116883116883116 46.36363636363637 94.70614625392552
PRFA performance for  NUM nan 0.0 nan 99.77272727272727
PRFA performance for  PRON nan 0.0 nan 99.68609865470852
PRFA performance for  NOUN 82.52873563218391 38.76889848812095 52.75532696546657 90.62545560577344
PRFA performance for  INTJ nan 0.0 nan 98.74213836477988
PRFA performance for  ADP 86.74080410607357 

/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:350: RuntimeWarning: invalid value encountered in double_scalars
  precision = 100 * grid[1, 1] / np.sum(grid[1])
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:351: RuntimeWarning: invalid value encountered in double_scalars
  recall = 100 * grid[1, 1] / np.sum(grid[:, 1])


Starting epoch 4
Starting epoch 5
------------------------------
total_eval_loss.shape torch.Size([])
PRFA performance for  SYM nan nan nan 100.0
PRFA performance for  PROPN 71.42857142857143 22.727272727272727 34.48275862068965 98.98612593383137
PRFA performance for  PUNCT 100.0 80.0 88.88888888888889 99.97405966277562
PRFA performance for  CCONJ nan nan nan 100.0
PRFA performance for  PART 67.03296703296704 62.88659793814433 64.8936170212766 94.1952506596306
PRFA performance for  ADJ 53.018372703412076 58.72093023255814 55.724137931034484 90.0093370681606
PRFA performance for  DET 87.25868725868726 90.4 88.8015717092338 98.39028523016097
PRFA performance for  ADV 72.72727272727273 46.753246753246756 56.91699604743083 95.10991475998206
PRFA performance for  NUM nan 0.0 nan 99.77272727272727
PRFA performance for  PRON nan 0.0 nan 99.68609865470852
PRFA performance for  NOUN 69.33962264150944 63.49892008639309 66.29086809470124 91.28152791952179
PRFA performance for  INTJ nan 0.0 nan 98

In [25]:
print(val_loss)
RNNseq_model

[tensor(0.1788, device='cuda:0'), tensor(0.1123, device='cuda:0'), tensor(0.0912, device='cuda:0'), tensor(0.0866, device='cuda:0')]


RNNSequenceModel(
  (rnn): LSTM(1324, 300, batch_first=True, bidirectional=True)
  (output_projection): Linear(in_features=600, out_features=2, bias=True)
  (dropout_on_input_to_LSTM): Dropout(p=0.5, inplace=False)
  (dropout_on_input_to_linear_layer): Dropout(p=0.1, inplace=False)
)

In [26]:
"""
for additional training
"""
rnn_optimizer = optim.Adam(RNNseq_model.parameters(), lr=0.0001)
for epoch in range(10):
    print("Starting epoch {}".format(epoch + 1))
    for (__, example_text, example_lengths, labels) in train_dataloader_vua:
        example_text = Variable(example_text)
        example_lengths = Variable(example_lengths)
        labels = Variable(labels)
        if using_GPU:
            example_text = example_text.cuda()
            example_lengths = example_lengths.cuda()
            labels = labels.cuda()
        # predicted shape: (batch_size, seq_len, 2)
        predicted = RNNseq_model(example_text, example_lengths)
        batch_loss = loss_criterion(predicted.view(-1, 2), labels.view(-1))
        rnn_optimizer.zero_grad()
        batch_loss.backward()
        rnn_optimizer.step()
        num_iter += 1
        # Calculate validation and training set loss and accuracy every 200 gradient updates
        if num_iter % 200 == 0:
            avg_eval_loss, performance_matrix = evaluate(idx2pos, val_dataloader_vua, RNNseq_model,
                                                         loss_criterion, using_GPU)
            val_loss.append(avg_eval_loss)
            val_f1s.append(performance_matrix[:, 2])
            print("Iteration {}. Validation Loss {}.".format(num_iter, avg_eval_loss))

#             avg_eval_loss, performance_matrix = evaluate(idx2pos, train_dataloader_vua, RNNseq_model,
#                                                          loss_criterion, using_GPU)
#             train_loss.append(avg_eval_loss)
#             train_f1s.append(performance_matrix[:, 2])
#             print("Iteration {}. Training Loss {}.".format(num_iter, avg_eval_loss))
#             comparable.append(get_performance())

print("Training done!")

Starting epoch 1


/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:217: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_text = Variable(eval_text, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:218: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_lengths = Variable(eval_lengths, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:219: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_labels = Variable(eval_labels, volatile=True)


------------------------------
total_eval_loss.shape torch.Size([])
PRFA performance for  SYM nan nan nan 100.0
PRFA performance for  PROPN 50.0 22.727272727272727 31.25 98.82604055496265
PRFA performance for  PUNCT 80.0 80.0 80.0 99.94811932555123
PRFA performance for  CCONJ nan nan nan 100.0
PRFA performance for  PART 62.5 72.16494845360825 66.98564593301435 93.93139841688654
PRFA performance for  ADJ 58.11209439528024 57.26744186046512 57.68667642752562 91.005291005291
PRFA performance for  DET 87.890625 90.0 88.93280632411067 98.41852584015815
PRFA performance for  ADV 64.75409836065573 51.298701298701296 57.246376811594196 94.70614625392552
PRFA performance for  NUM nan 0.0 nan 99.77272727272727
PRFA performance for  PRON nan 0.0 nan 99.68609865470852
PRFA performance for  NOUN 69.97607655502392 63.17494600431966 66.40181611804768 91.36900422802158
PRFA performance for  INTJ nan 0.0 nan 98.74213836477988
PRFA performance for  ADP 85.31468531468532 82.00149365197909 83.625285605483

/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:350: RuntimeWarning: invalid value encountered in double_scalars
  precision = 100 * grid[1, 1] / np.sum(grid[1])
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:351: RuntimeWarning: invalid value encountered in double_scalars
  recall = 100 * grid[1, 1] / np.sum(grid[:, 1])


Starting epoch 2
Starting epoch 3
------------------------------
total_eval_loss.shape torch.Size([])
PRFA performance for  SYM nan nan nan 100.0
PRFA performance for  PROPN 50.0 22.727272727272727 31.25 98.82604055496265
PRFA performance for  PUNCT 80.0 80.0 80.0 99.94811932555123
PRFA performance for  CCONJ nan nan nan 100.0
PRFA performance for  PART 65.3061224489796 65.97938144329896 65.64102564102564 94.10729991204926
PRFA performance for  ADJ 60.204081632653065 51.45348837209303 55.4858934169279 91.16090880796763
PRFA performance for  DET 87.59689922480621 90.4 88.97637795275591 98.41852584015815
PRFA performance for  ADV 66.66666666666667 49.35064935064935 56.71641791044776 94.79587258860475
PRFA performance for  NUM nan 0.0 nan 99.77272727272727
PRFA performance for  PRON nan 0.0 nan 99.68609865470852
PRFA performance for  NOUN 74.01055408970976 60.58315334773218 66.6270783847981 91.80638577052048
PRFA performance for  INTJ nan 0.0 nan 98.74213836477988
PRFA performance for  AD

In [27]:
"""
test on genres by POS tags
"""
print("**********************************************************")
print("Evalutation on test set: ")

raw_test_vua = []
with open(data_dir + 'VUAsequence/VUA_seq_formatted_test.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        # txt_id	sen_ix	sentence	label_seq	pos_seq	labeled_sentence	genre
        pos_seq = ast.literal_eval(line[4])
        label_seq = ast.literal_eval(line[3])
        assert(len(pos_seq) == len(label_seq))
        assert(len(line[2].split()) == len(pos_seq))
        raw_test_vua.append([line[2], label_seq, pos_seq])
print('number of examples(sentences) for test_set ', len(raw_test_vua))

for i in range(len(raw_test_vua)):
    raw_test_vua[i][2] = index_sequence(pos2idx, raw_test_vua[i][2])

elmos_test_vua = h5py.File(elmo_dir + 'VUA_test.hdf5', 'r')
# raw_train_vua: sentence, label_seq, pos_seq
# embedded_train_vua: embedded_sentence, pos, labels
embedded_test_vua = [[embed_indexed_sequence(example[0], example[2], word2idx,
                                      glove_embeddings, elmos_test_vua, suffix_embeddings),
                       example[2], example[1]]
                      for example in raw_test_vua]

# Separate the input (embedded_sequence) and labels in the indexed train sets.
# embedded_train_vua: embedded_sentence, pos, labels
test_dataset_vua = TextDataset([example[0] for example in embedded_test_vua],
                              [example[1] for example in embedded_test_vua],
                              [example[2] for example in embedded_test_vua])

# Set up a DataLoader for the test dataset
test_dataloader_vua = DataLoader(dataset=test_dataset_vua, batch_size=batch_size,
                              collate_fn=TextDataset.collate_fn)

print("Tagging model performance on VUA test set by POS tags: regardless of genres")
avg_eval_loss, performance_matrix = evaluate(idx2pos, test_dataloader_vua, RNNseq_model, loss_criterion, using_GPU)

**********************************************************
Evalutation on test set: 
number of examples(sentences) for test_set  2694
Tagging model performance on VUA test set by POS tags: regardless of genres


/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:217: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_text = Variable(eval_text, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:218: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_lengths = Variable(eval_lengths, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:219: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_labels = Variable(eval_labels, volatile=True)


------------------------------
total_eval_loss.shape torch.Size([])
PRFA performance for  SYM nan nan nan 100.0
PRFA performance for  PROPN 44.44444444444444 12.903225806451612 20.0 98.32020997375328
PRFA performance for  PUNCT 0.0 0.0 nan 99.9213217938631
PRFA performance for  CCONJ nan nan nan 100.0
PRFA performance for  PART 57.241379310344826 55.70469798657718 56.462585034013614 91.25085440874915
PRFA performance for  ADJ 62.15596330275229 50.09242144177449 55.47594677584442 89.02900378310214
PRFA performance for  DET 89.90825688073394 93.55608591885442 91.69590643274854 98.27586206896552
PRFA performance for  ADV 76.59574468085107 59.01639344262295 66.66666666666667 95.75596816976127
PRFA performance for  NUM nan nan nan 100.0
PRFA performance for  PRON nan 0.0 nan 99.84829329962074
PRFA performance for  NOUN 71.1063372717508 51.27807900852053 59.58595859585959 89.54354913833255
PRFA performance for  INTJ nan 0.0 nan 99.49748743718592
PRFA performance for  ADP 89.38377059182429 87

/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:350: RuntimeWarning: invalid value encountered in double_scalars
  precision = 100 * grid[1, 1] / np.sum(grid[1])
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:351: RuntimeWarning: invalid value encountered in double_scalars
  recall = 100 * grid[1, 1] / np.sum(grid[:, 1])
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:352: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2 * precision * recall / (precision + recall)


In [ ]:
avg_eval_loss, performance_matrix = evaluate(idx2pos, test_dataloader_vua, RNNseq_model, loss_criterion, using_GPU)

array([[  0.        ,   0.        ,          nan,  99.82300885],
       [         nan,   0.        ,          nan,  99.49748744],
       [         nan,          nan,          nan, 100.        ],
       [         nan,          nan,          nan, 100.        ],
       [         nan,          nan,          nan, 100.        ],
       [ 71.15727003,  63.5400106 ,  67.13325868,  88.10777958],
       [ 74.12935323,  61.06557377,  66.96629213,  95.66755084],
       [ 90.        ,  94.51073986,  92.20023283,  98.3729966 ],
       [  0.        ,   0.        ,          nan,  99.92132179],
       [         nan,          nan,          nan, 100.        ],
       [ 58.33333333,  22.58064516,  32.55813953,  98.47769029],
       [ 88.44765343,  88.0239521 ,  88.23529412,  92.60377358],
       [ 55.48387097,  57.71812081,  56.57894737,  90.97744361],
       [ 65.45454545,  53.23475046,  58.71559633,  89.78562421],
       [ 69.79472141,  55.30596437,  61.71132239,  89.68327899]])

In [74]:
"""
write the test prediction on the VUA-verb to a file: sequence prediction
read and extract to get a comparabel performance on VUA-verb test set.
"""
def get_comparable_performance_test():
    result = write_predictions(raw_test_vua, test_dataloader_vua, RNNseq_model, using_GPU, data_dir+'VUAsequence/VUA_seq_formatted_test.csv')
    f = open('predictions/vua_seq_test_predictions_LSTMsequence_vua.csv', 'w')
    writer = csv.writer(f)
    writer.writerows(result)
    f.close()

    get_performance_VUAverb_test(data_dir)
    get_performance_VUA_test(data_dir)

get_comparable_performance_test()

/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:303: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_text = Variable(eval_text, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:304: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_lengths = Variable(eval_lengths, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:305: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_labels = Variable(eval_labels, volatile=True)


FileNotFoundError: ignored

In [57]:
g = open('predictions/blank.csv', 'w')

In [70]:
!git commit -m "new predictions directory for vua evaluation results"

[main 7097ba6] new predictions directory for vua evaluation results
 2 files changed, 2695 insertions(+)
 create mode 100644 predictions/blank.csv
 create mode 100644 predictions/vua_seq_test_predictions_LSTMsequence_vua.csv


In [76]:
!git add

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   notebooks/sequence/replicate_vua_seq_model_for_classification.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
